In [ ]:
!pip install google-ai-generativelanguage==0.6.15
!pip install -qU "langchain[google-genai]"
!pip install dateparser
!pip install langchain-community

  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl (1.3 MB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.18
    Uninstalling google-ai-generativelanguage-0.6.18:
      Successfully uninstalled google-ai-generativelanguage-0.6.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.7 requires google-ai-generativelanguage<0.7.0,>=0.6.18, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [ ]:
!pip show langchain
!pip show google-generativeai
!pip show google-ai-generativelanguage
!pip show dateparser
!pip show langchain-community

Name: langchain
Version: 0.3.26
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: langchain-community
Name: google-generativeai
Version: 0.8.5
Summary: Google Generative AI High level API client library and tools.
Home-page: https://github.com/google/generative-ai-python
Author: Google LLC
Author-email: googleapis-packages@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: google-ai-generativelanguage, google-api-core, google-api-python-client, google-auth, protobuf, pydantic, tqdm, typing-extensions
Required-by: 
Name: google-ai-generativelanguage
Version: 0.6.18
Summary: Google Ai Generativelanguage API client library
Home-page: https://github.com/googleapis/google-cloud-python/tree/main/packages/google-ai-generativelangu

In [ ]:
from datetime import datetime
from typing import List, Optional
from pydantic import BaseModel, Field
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.llms import HuggingFacePipeline
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from dateparser import parse as parse_date
import json
import os
import getpass
from langchain.chat_models import init_chat_model

# === Step 1: Define the simulated tool ===
def add_event_to_calendar(title: str, date: str, time: str, location: Optional[str] = None, attendees: Optional[List[str]] = None) -> str:
    msg = f"Event '{title}' on {date} at {time}"
    if location:
        msg += f" in {location}"
    if attendees:
        msg += f" with {', '.join(attendees)}"
    msg += " successfully added."
    return msg

# === Step 2: Define Pydantic schema for output parsing ===
# Pydantic model for structured event data
class EventDetails(BaseModel):
    title: str = Field(..., description="Title of the event")
    date: str = Field(..., description="Date of the event in YYYY-MM-DD format")
    time: str = Field(..., description="Time of the event in HH:MM format")
    location: Optional[str] = Field(None, description="Location of the event")
    attendees: Optional[List[str]] = Field(None, description="List of people attending the event")

parser = PydanticOutputParser(pydantic_object=EventDetails)

# === Step 3: Prompt for structured extraction ===
today = datetime.now().strftime("%Y-%m-%d")

prompt = PromptTemplate(
    template="""
You are an assistant that extracts event information.
Extract the following: title, date, time, location (optional), attendees (optional) in a structured format.
Use current date {today} to resolve relative terms like "tomorrow", "next Monday", etc.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

{format_instructions}

Input: {input}
""",
    input_variables=["input", "today"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# === Step 4: Date resolution function ===
def resolve_date(date_str: str, today: str = today) -> str:
    parsed = parse_date(date_str, settings={"RELATIVE_BASE": datetime.strptime(today, "%Y-%m-%d")})
    return parsed.strftime("%Y-%m-%d") if parsed else date_str

# === Step 5: Load the Gemini model ===
if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")
llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

# === Step 6: LangChain Tool Wrapper ===
def tool_func(input: str) -> str:
    prompt_value = prompt.format_prompt(input=input, today=today)
    output = llm(prompt_value.to_messages())
    parsed_event = parser.parse(output.content)
    parsed_event.date = resolve_date(parsed_event.date, today=today)
    return add_event_to_calendar(
        title=parsed_event.title,
        date=parsed_event.date,
        time=parsed_event.time,
        location=parsed_event.location,
        attendees=parsed_event.attendees
    )

# === Step 7: Register the tool ===
event_tool = Tool(
    name="add_event_to_calendar",
    func=tool_func,
    description="Use this tool to add an event to the calendar. Requires title (str), date (YYYY-MM-DD), time (HH:MM). Optional: location (str), attendees (list of str)."
)

# === Step 8: Build the agent ===
prefix = f"""You are an event scheduling assistant. Today's date is {today}.

You have access to the following tool:

add_event_to_calendar(input: str) -> str -
    Use this tool to add an event to the calendar.
    Requires title (str), date (YYYY-MM-DD), time (HH:MM).
    Optional: location (str), attendees (list of str).

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [add_event_to_calendar]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Only respond to the given question. Stop after the Final Answer.

Begin!"""

agent = initialize_agent(
    tools=[event_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    agent_kwargs={"prefix": prefix}
)

# === Step 9: Run the agent ===
# if __name__ == "__main__":
import time
test_input = "Schedule a team meeting for tomorrow at 3 PM in Conference Room A with Alice and Bob about Q3 planning"
print(f"User: {test_input}")
start = time.time()
result = agent.invoke(test_input)
end = time.time()
print("\nAssistant:", result)
print(f"Time taken: {end - start:.2f} seconds")


Enter API key for Google Gemini: ··········


/tmp/ipython-input-2-1248565122.py:111: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


User: Schedule a team meeting for tomorrow at 3 PM in Conference Room A with Alice and Bob about Q3 planning


> Entering new AgentExecutor chain...
I need to schedule a team meeting for tomorrow (2025-07-13) at 3 PM in Conference Room A with Alice and Bob about Q3 planning.
Action: add_event_to_calendar
Action Input: {"title": "Q3 Planning Team Meeting", "date": "2025-07-13", "time": "15:00", "location": "Conference Room A", "attendees": ["Alice", "Bob"]}

/tmp/ipython-input-2-1248565122.py:68: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output = llm(prompt_value.to_messages())



Observation: Event 'Q3 Planning Team Meeting' on 2025-07-13 at 15:00 in Conference Room A with Alice, Bob successfully added.
Thought:I have successfully added the event to the calendar.
Final Answer: OK

> Finished chain.

Assistant: {'input': 'Schedule a team meeting for tomorrow at 3 PM in Conference Room A with Alice and Bob about Q3 planning', 'output': 'OK'}
Time taken: 2.72 seconds
